<!-- Introduction -->
<h2><b>Introduction</b></h2>

<p>In procurement and supplier selection, organizations face the challenge of evaluating multiple vendors based on various criteria such as <b>price, delivery cost, bid price, and extra charges</b>. Choosing the right supplier is crucial for optimizing costs, ensuring quality, and maintaining operational efficiency. However, selecting the best vendor is a complex decision-making process that involves analyzing multiple factors simultaneously.</p>

<p>To address this challenge, we leverage <b>machine learning techniques</b> to identify and assign <b>criteria weights</b> to key evaluation factors. By analyzing historical data, we can determine which factors contribute most to supplier selection and use these insights to make more objective and data-driven decisions.</p>

<p>Our approach begins with <b>preprocessing and transforming</b> the dataset to address <b>data inconsistencies</b> and optimize <b>feature engineering</b>. Next, we develop <b>machine learning models</b> to analyze <b>feature importance</b>, allowing us to derive the weight of each criterion in the decision-making process.</p>

<p>Once the criteria weights are determined, we proceed to <b>rank vendors</b> using <b>Multi-Criteria Decision Making (MCDM) techniques</b>. These techniques, including <b>TOPSIS, VIKOR and AHP</b>, help us evaluate suppliers by considering multiple weighted criteria, ensuring a fair and systematic ranking process.</p>

<p>Through this comprehensive analysis, we provide <b>valuable insights</b> into supplier evaluation, enabling better procurement decisions based on <b>quantitative and objective methods</b>. This data-driven approach enhances transparency, efficiency, and effectiveness in vendor selection.</p>


<!-- Installing Required Libraries -->
<h2><b>Installing Required Libraries</b></h2>
<p>Before performing any analysis, we need to install the necessary libraries:</p>
<ul>
    <li><b><code>pandas</code></b>: Essential for data manipulation and analysis.</li>
    <li><b><code>odfpy</code></b>: Required for handling OpenDocument Format (ODF) files.</li>
    <li><b><code>scikit-learn</code></b>: For machine learning models and data preprocessing.</li>

</ul>


In [54]:
!pip install pandas odfpy scikit-learn



<!-- Importing Dependencies -->
<h2><b>Importing Dependencies</b></h2>
<p>We now import the key libraries needed for data processing, modeling, and visualization:</p>
<ul>
    <li><b><code>pandas</code></b> and <b><code>numpy</code></b>: For handling structured data and numerical computations.</li>
    <li><b><code>sklearn</code></b>: Provides tools for preprocessing, model training, and evaluation.</li>
    <li><b><code>lightgbm</code></b>: A high-performance gradient boosting framework.</li>
    <li><b><code>seaborn</code></b> and <b><code>matplotlib</code></b>: Useful for visualizing data trends.</li>
    <li><b>Additional Libraries:</b> Depending on our specific needs, we may utilize other libraries for data processing, visualization, or modeling.</li>
</ul>


In [55]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.linalg import eig
from itertools import combinations

<!-- Loading the Dataset -->
<h2><b>Loading the Dataset</b></h2>
<p>We load the <code>.ods</code> file into a pandas DataFrame, allowing us to manipulate and analyze the data efficiently.</p>


In [56]:
data = pd.read_excel('/content/supplier_data.ods', engine='odf')


<!-- Renaming Columns for Consistency -->
<h2><b>Renaming Columns for Consistency</b></h2>
<p>To ensure clarity and uniformity, we correct any typos and inconsistencies in column names.</p>


In [57]:
data = data.rename(columns={
    'technecal_acceptance': 'technical_acceptance',
    'extra_charage': 'extra_charge',
    'items_offred': 'items_offered',
    'items_reqested': 'items_requested',
    'delivery_t': 'delivery_time'
})


<h2><b>Removing Redundant and Irrelevant Columns</b></h2>
<p>Some columns do not contribute to meaningful analysis and should be removed. Before dropping them, we check their unique values to confirm redundancy.</p>


In [58]:
cols_to_remove = ['bidreceive', 'extra_cha2', 'warranty', 'matl_orig', 'disc_amt', 'order number', 'no_of_item', 'items_requested']
redundant_cols = ['bidreceive', 'extra_cha2', 'warranty', 'matl_orig', 'disc_amt']

result = {col: data[col].value_counts() for col in redundant_cols}

print("\nCounts of unique values:\n")
for col, counts in result.items():
    print(counts.to_string(index=True))
    print()

data.drop(columns=cols_to_remove, inplace=True)
print("Specified columns have been removed.")



Counts of unique values:

bidreceive
-   -    20347

extra_cha2
0    20347

warranty
1 YEAR    1

matl_orig
BULACAN         1
BULACAN TEST    1

disc_amt
0.00      20346
386.23        1

Specified columns have been removed.


<h2>Filtering Out Invalid Bids</h2>
<p>Any rows where the <code>bid_price</code> is zero indicate invalid or incomplete bids and should be removed.</p>


In [59]:
data = data[data['bid_price'] != 0]
print(f"Number of rows after removing zero bid prices: {data.shape[0]}")


Number of rows after removing zero bid prices: 7915


<h2>Handling Corrupt and Missing Date Values</h2>
<p>Certain date fields contain invalid entries, which we need to filter out before performing any date-based calculations.</p>


In [60]:
data=data[data['offer_date']!='###############################################################################################################################################################################################################################################################']
data=data[data['offer_date']!='  -   -']
data=data[data['offervalid']!='###############################################################################################################################################################################################################################################################']
data=data[data['offervalid']!='  -   -']

<h2>Standardizing Date Format</h2>
<p>To maintain consistency, we reformat date columns into <code>dd/mm/yyyy</code> format and ensure chronological correctness.</p>


In [61]:
data[['offer_date', 'offervalid']] = data[['offer_date', 'offervalid']].astype(str)

data['offer_date'] = (
    data['offer_date'].str.slice(8, 10) + '/' +
    data['offer_date'].str.slice(5, 7) + '/' +
    data['offer_date'].str.slice(0, 4)
)

data['offervalid'] = (
    data['offervalid'].str.slice(8, 10) + '/' +
    data['offervalid'].str.slice(5, 7) + '/' +
    data['offervalid'].str.slice(0, 4)
)

data['offervalid'] = data.apply(
    lambda row: row['offervalid'][:-4] + row['offer_date'][-4:], axis=1
)


<h2>Calculating Bid Validity Duration</h2>
<p>We compute the number of days between <code>offer_date</code> and <code>offervalid</code> to create a new feature, <code>validity_days</code>, which indicates how long the bid remains valid.</p>


In [62]:
days_between = [
    abs((int(offervalid.split('/')[2]) - int(offer_date.split('/')[2])) * 365 +
        (int(offervalid.split('/')[1]) - int(offer_date.split('/')[1])) * 30 +
        (int(offervalid.split('/')[0]) - int(offer_date.split('/')[0])))
    for offer_date, offervalid in zip(data['offer_date'], data['offervalid'])
]
data['validity_days'] = days_between
data.drop(columns=['offer_date','offervalid'],inplace=True)


<div>
    <h2>Handling Zero USD Prices</h2>
    <p>The output reveals that our dataset includes rows where <code>total_amount</code> has a valid value, but <code>usd_price</code> is zero. To resolve this:</p>
    <ul>
        <li>We computed the <b>conversion rate</b> for each currency to USD.</li>
        <li>We applied this rate to convert <code>total_amount</code> into <code>usd_price</code> for rows where <code>usd_price</code> was initially zero.</li>
    </ul>
    <p>By standardizing all prices in USD, we enhance data consistency, ensuring more reliable vendor comparisons and accurate cost analysis.</p>
</div>


In [63]:
conversion_rates = (
    data.dropna(subset=['usd_price'])
    .groupby('currency', group_keys=False)
    .apply(lambda x: x.iloc[0]['usd_price'] / x.iloc[0]['total_amount'], include_groups=False)
    .to_dict()
)


missing_usd_price_condition = data['usd_price'] == 0

data.loc[missing_usd_price_condition, 'usd_price'] = data.loc[
    missing_usd_price_condition
].apply(lambda row: row['total_amount'] * conversion_rates.get(row['currency'], 0), axis=1)

data = data[data['usd_price'] != 0]


<div>
    <h2>Transforming Categorical Columns</h2>
    <p>
        To make our data suitable for modeling, we must convert categorical columns into numerical representations.
        Since machine learning models cannot process categorical data directly, we first analyze the number of unique
        values in each categorical column before applying transformations.
    </p>
</div>


In [64]:
categorical_cols=['currency', 'delivery_place', 'technical_acceptance',
                  'action', 'delivery_term', 'payment_term']

result = {col: data[col].value_counts() for col in categorical_cols}
print("Counts of unique values:")
for col, counts in result.items():
    print(counts.to_string())
    print()

Counts of unique values:
currency
EUR    4544
GBP    2262
USD     681
CAD      32
LD       26
CHF      17
JPY      12
SEK       5
DKK       1

delivery_place
Libya    4245
BREGA    1404

technical_acceptance
True     5729
False    1853

action
F    6461
P     800
N      13
D       7
Q       5
I       5

delivery_term
1    5361
2     643
b     371
5     257
3     121
6     109
c      94
4      67
f      52
7      49
9      32
8      29
d      11
A       7
e       7

payment_term
2    4722
1    1211
7     294
a     282
5     219
8     165
3     158
9     153
4      82
6      74
b      38
d       1



<h2>Numerical Encoding for Categorical Columns</h2>
<p>To make categorical features suitable for machine learning models, we applied numerical encoding based on domain-specific reasoning.</p>

<h3>1. Currency Mapping</h3>
<ul>
    <li>Currencies with lower USD equivalents receive higher values.</li>
    <li>Since our company operates in Libya, we prioritize currencies that reduce our USD expenses.</li>
</ul>

<h3>2. Delivery Place Mapping</h3>
<ul>
    <li>Mapped according to extra delivery charges.</li>
    <li><strong>BREGA</strong> is assigned a higher value due to lower additional costs.</li>
</ul>

<h3>3. Action Mapping</h3>
<ul>
    <li>Encodes actions based on desirability:</li>
    <li><strong>F (Fulfilled)</strong> → Higher value (positive outcome).</li>
    <li><strong>D (Declined)</strong> → Lower value (negative outcome).</li>
</ul>

<h3>4. Technical Acceptance Mapping</h3>
<ul>
    <li><strong>Accepted (True)</strong> → <code>2.0</code></li>
    <li><strong>Rejected (False)</strong> → <code>1.0</code></li>
</ul>

<p>This encoding ensures our models can interpret categorical variables effectively while maintaining business relevance.</p>


In [65]:
currency_mapping = {'EUR': 2.0, 'GBP': 1.0, 'USD': 4.0, 'CAD': 3.0, 'LD': 6.0, 'CHF': 5.0, 'JPY': 9.0, 'SEK': 8.0, 'DKK': 7.0}
deliveryplace_mapping = {'Libya': 1.0, 'BREGA': 2.0}
action_mapping = {'F': 6.0, 'P': 5.0, 'N': 2.0, 'D': 1.0, 'Q': 4.0, 'I': 3.0}

data = data.copy()

data['delivery_place'] = data['delivery_place'].map(deliveryplace_mapping)
data['currency'] = data['currency'].map(currency_mapping)
data['technical_acceptance'] = data['technical_acceptance'].map({True: 2.0, False: 1.0})
data['action'] = data['action'].map(action_mapping)



<div>
    <h2>Frequency Encoding for Categorical Columns</h2>
    <p>For some categorical columns, like <strong>delivery_term</strong> and <strong>payment_term</strong>, we use frequency encoding. This means:</p>
    <ul>
        <li>Each category is replaced with its frequency (how often it appears in the dataset).</li>
        <li>This approach assumes that more frequent categories are more important or relevant.</li>
        <li>Frequency encoding is particularly useful when there’s no inherent order or hierarchy in the categories, allowing us to capture their prevalence in the data.</li>
    </ul>
</div>


In [66]:
data = data.copy()

data.loc[:, 'delivery_term'] = data['delivery_term'].astype(str)
delivery_term_encoding = data['delivery_term'].value_counts(normalize=True)
data.loc[:, 'delivery_term'] = data['delivery_term'].map(delivery_term_encoding)

data.loc[:, 'payment_term'] = data['payment_term'].astype(str)
payment_term_encoding = data['payment_term'].value_counts(normalize=True)
data.loc[:, 'payment_term'] = data['payment_term'].map(payment_term_encoding)


<div>
    <h2>Handling Missing Values</h2>
    <p>Missing values can disrupt our analysis and modeling. Here, we handle them dynamically:</p>
    <ul>
        <li>For each missing value, we calculate the average value for that vendor (<strong>vendor code</strong>).</li>
        <li>If no data is available for the vendor, we mark the value as <code>NaN</code> and later remove those rows.</li>
        <li>This approach ensures that missing values are filled in a way that reflects vendor-specific patterns, making the imputation more accurate and meaningful.</li>
    </ul>
</div>


In [67]:
def impute_missing_values_dynamic(df, column, group_column='vendor code'):
    vendor_avg = df.groupby(group_column)[column].mean().round().astype(float)
    counts = df.groupby(group_column)[column].count()
    sums = df.groupby(group_column)[column].sum()

    def impute_value(row):
        if pd.isnull(row[column]):
            group = row[group_column]
            if counts[group] == 0:
                return 'nan'
            avg = sums[group] / counts[group]
            avg_rounded = float(round(avg))
            counts[group] += 1
            sums[group] += avg_rounded
            return avg_rounded
        return row[column]

    df[column] = df.apply(impute_value, axis=1)
    return df

columns_to_impute = ['action', 'currency', 'delivery_place']
for cols in columns_to_impute:
    data = impute_missing_values_dynamic(data, cols)

data = data[data['delivery_place'] != 'nan']
data = data[data['action'] != 'nan']

<div>
    <h2>Standardizing Monetary Values</h2>
    <p>To ensure consistency, we convert all monetary values to USD:</p>
    <ul>
        <li>Using the previously calculated conversion rates, we transform <strong>extra_charge</strong> and <strong>bid_price</strong> into USD equivalents.</li>
        <li>This standardization allows us to compare prices across different currencies directly.</li>
        <li>This step is crucial for accurate cost analysis and vendor comparisons.</li>
    </ul>
</div>


In [68]:
mapped_conversion_rates = {currency_mapping[key]: conversion_rates[key] for key in conversion_rates if key in currency_mapping}

def apply_conversion_rate(row, column_name):
    currency = row['currency']
    if currency in mapped_conversion_rates:
        conversion_rate = mapped_conversion_rates[currency]
        return row[column_name] * conversion_rate
    else:
        return row[column_name]

data['extra_charge_usd'] = data.apply(lambda row: apply_conversion_rate(row, 'extra_charge'), axis=1)
data['bidprice_usd'] = data.apply(lambda row: apply_conversion_rate(row, 'bid_price'), axis=1)

<div>
    <h2>K-Nearest Neighbors (KNN) Imputation</h2>
    <p>For columns like <strong>delivery_term</strong> and <strong>payment_term</strong>, we use K-Nearest Neighbors (KNN) imputation:</p>
    <ul>
        <li>KNN imputation estimates missing values based on the values of the nearest neighbors in the dataset.</li>
        <li>This method is particularly useful when the missing values are related to other features in the dataset.</li>
        <li>By using KNN, we ensure that the imputed values are consistent with the overall patterns in the data.</li>
    </ul>
</div>


In [69]:
imputer_data = data[['delivery_term', 'payment_term']]
imputer = KNNImputer(n_neighbors=5)
imputed_data = imputer.fit_transform(imputer_data)

data['delivery_term'] = imputed_data[:, 0]
data['payment_term'] = imputed_data[:, 1]

<div>
    <h2>Feature Engineering</h2>
    <p>Feature engineering is where we create new features to improve our model’s performance:</p>
    <ul>
        <li><strong>usdprice_after_discount:</strong> The final price after applying the discount, which is our target variable.</li>
        <li><strong>Per-item metrics:</strong> We calculate <em>bid_price_per_item</em>, <em>discount_per_item</em>, and <em>delivery_cost_per_item</em> to normalize costs based on the number of items.</li>
        <li><strong>Validity periods:</strong> We convert <em>validity_days</em> into weeks and months for easier interpretation.</li>
        <li><strong>Vendor statistics:</strong> We compute vendor-specific metrics like average bid price, standard deviation, and fulfillment rate to capture vendor performance.</li>
    </ul>
    <p>These new features provide richer insights and help the model better understand the relationships in the data.</p>
</div>


In [70]:
# Feature engineering
data['usdprice_after_discount'] = data['usd_price'] * (1 - data['discount'] / 100)

data['bid_price_per_item'] = data['bidprice_usd'] / data['items_offered'].replace(0, np.nan)
data['discount_per_item'] = data['discount'] / data['items_offered'].replace(0, np.nan)
data['delivery_cost_per_item'] = data['extra_charge_usd'] / data['items_offered'].replace(0, np.nan)

data['bid_price_per_item'] = data['bid_price_per_item'].fillna(0)
data['discount_per_item'] = data['discount_per_item'].fillna(0)
data['delivery_cost_per_item'] = data['delivery_cost_per_item'].fillna(0)

data['validity_weeks'] = data['validity_days'] / 7
data['validity_months'] = data['validity_days'] / 30

# Vendor-specific statistics
vendor_stats = data.groupby('vendor code').agg({
    'bidprice_usd': ['mean', 'std'],
    'action': lambda x: (x == 'F').mean()
}).reset_index()

vendor_stats.columns = ['vendor code', 'avg_bid_price', 'std_bid_price', 'fulfillment_rate']
data = data.merge(vendor_stats, on='vendor code', how='left')

data['avg_bid_price'] = data['avg_bid_price'].fillna(data['bidprice_usd'].mean())
data['std_bid_price'] = data['std_bid_price'].fillna(0)
data['fulfillment_rate'] = data['fulfillment_rate'].fillna(0)

The warnings module in Python helps control warning messages. Here, we use it to suppress unnecessary FutureWarnings and DeprecationWarnings, which indicate upcoming library changes but don't affect model correctness. This keeps the output clean and focused on meaningful results.

In [71]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)


<div>
    <h2>Model Training & Evaluation</h2>
    <p>We train multiple regression models and evaluate their performance using cross-validation. Our process includes:</p>
    <ul>
        <li>Scaling features using <b>MinMaxScaler</b>.</li>
        <li>Training models like RandomForest, ExtraTrees, AdaBoost, LGBM, and Ridge regression.</li>
        <li>Computing mean absolute error (MAE) for both training and validation sets.</li>
        <li>Deriving feature importances for models that support it.</li>
    </ul>
    <p>This approach ensures robust model selection while providing insights into feature relevance.</p> <h3>Scaling & Cross-Validation</h3>
    <ul>
        <li>Data is scaled using <b>MinMaxScaler</b> for consistency.</li>
        <li><b>5-fold cross-validation</b> evaluates models based on:
            <ul>
                <li><b>Mean Absolute Error (MAE)</b> – measures training & validation performance.</li>
                <li><b>Average Error per Row</b> – tracks error impact based on dataset size.</li>
            </ul>
        </li>
    </ul> <h3>Feature Importance Analysis</h3>
    <ul>
        <li>Tree-based models provide <b>feature importances</b>.</li>
        <li>Ridge Regression uses <b>absolute coefficients</b>.</li>
        <li>Models without importance scores assume equal feature contributions.</li>
        <li>Final rankings highlight key factors affecting price variations.</li>
    </ul>
</div>


In [72]:
X = data.drop(['vendor code', 'usdprice_after_discount'], axis=1)
y = data['usdprice_after_discount']

n_rows = len(data)

models = {
    'RandomForest': RandomForestRegressor(max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=150, random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42),
    'AdaBoostRegressor': AdaBoostRegressor(random_state=42),
    'LGBMRegressor': LGBMRegressor(n_estimators=100, learning_rate=0.1, verbose=-1, random_state=42),
    'Ridge': Ridge(alpha=1.0)
}

results = {}
scalar = MinMaxScaler()
feature_importances = {model_name: [] for model_name in models.keys()}

for name, model in models.items():
    pipeline = make_pipeline(scalar, model)
    cv_results = cross_validate(pipeline, X, y, cv=5,
                                scoring=['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2'],
                                return_train_score=True)

    train_mae = -cv_results['train_neg_mean_absolute_error'].mean()
    valid_mae = -cv_results['test_neg_mean_absolute_error'].mean()

    results[name] = {
        'train_mae': train_mae,
        'valid_mae': valid_mae,
        'train_avg_error_per_row': train_mae / n_rows,
        'valid_avg_error_per_row': valid_mae / n_rows
    }

    print(f"{name}")
    print(f"MAE-train: {results[name]['train_mae']:.2f}")
    print(f"MAE-valid: {results[name]['valid_mae']:.2f}")
    print(f"Average Error per Row - train: {results[name]['train_avg_error_per_row']:.6f}")
    print(f"Average Error per Row - valid: {results[name]['valid_avg_error_per_row']:.6f}")
    print()

    pipeline.fit(X, y)

    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        importances_normalized = importances / np.sum(importances)
        feature_importances[name] = importances_normalized

        df_imp = pd.DataFrame({
            'Feature': X.columns,
            'Importance': importances_normalized
        }).sort_values(by='Importance', ascending=False).reset_index(drop=True)
    elif hasattr(model, 'coef_'):
        importances = np.abs(model.coef_)
        importances_normalized = importances / np.sum(importances)
        feature_importances[name] = importances_normalized

        df_imp = pd.DataFrame({
            'Feature': X.columns,
            'Importance': importances_normalized
        }).sort_values(by='Importance', ascending=False).reset_index(drop=True)
    else:
        feature_importances[name] = [0] * len(X.columns)

df_feature_importances = pd.DataFrame(feature_importances, index=X.columns)
df_feature_importances = df_feature_importances.sort_values(by=list(models.keys()), ascending=False)


RandomForest
MAE-train: 4094.93
MAE-valid: 11646.18
Average Error per Row - train: 0.545119
Average Error per Row - valid: 1.550344

ExtraTrees
MAE-train: 0.00
MAE-valid: 4954.63
Average Error per Row - train: 0.000000
Average Error per Row - valid: 0.659562

AdaBoostRegressor
MAE-train: 17110.52
MAE-valid: 23299.11
Average Error per Row - train: 2.277759
Average Error per Row - valid: 3.101585

LGBMRegressor
MAE-train: 26364.13
MAE-valid: 29068.43
Average Error per Row - train: 3.509602
Average Error per Row - valid: 3.869599

Ridge
MAE-train: 28977.68
MAE-valid: 31305.42
Average Error per Row - train: 3.857519
Average Error per Row - valid: 4.167388



By leveraging Machine Learning models, we obtain the feature importance values for each attribute. These values serve as the weights for our criteria, guiding the decision-making process.







In [73]:
df_feature_importances.head(10)

,RandomForest,ExtraTrees,AdaBoostRegressor,LGBMRegressor,Ridge
usd_price,0.253876,0.301072,0.388517,0.260000,0.212301
delivery_cost_per_item,0.190413,0.043167,0.078709,0.090333,0.043471
bid_price,0.156071,0.147506,0.091448,0.071333,0.124427
extra_charge_usd,0.131033,0.098517,0.006878,0.051333,0.122506
total_amount,0.090611,0.128473,0.069632,0.054333,0.120607
bidprice_usd,0.069796,0.248162,0.174744,0.058000,0.178541
bid_price_per_item,0.063871,0.018579,0.041601,0.118333,0.087319
extra_charge,0.039640,0.007495,0.144832,0.096333,0.074261
payment_term,0.000829,0.000698,0.000000,0.018333,0.000102
validity_months,0.000711,0.001264,0.001054,0.000000,0.000051


<h1>Multi-Criteria Decision Making (MCDM)</h1> <p> Now that we have determined the criteria weights using feature importance values, we can proceed with constructing the decision matrix for vendor evaluation. This involves selecting the most relevant features and normalizing them appropriately. </p> <p> Features that positively impact vendor selection are scaled within the range [0,1], while cost-related attributes are transformed to reflect their inverse relationship with desirability. These weighted and normalized criteria form the foundation for applying MCDM techniques to rank vendors effectively. </p>

In [74]:
def prepare_decision_matrix(data, df_feature_importances):
    selected_features = df_feature_importances.loc[(df_feature_importances != 0).any(axis=1)].index.tolist()
    decision_matrix = data[['vendor code'] + selected_features].copy()

    benefit_criteria = ['usd_price', 'bid_price_per_item']
    cost_criteria = ['delivery_cost_per_item', 'extra_charge_usd']

    scaler = MinMaxScaler()
    decision_matrix[benefit_criteria] = scaler.fit_transform(decision_matrix[benefit_criteria])
    decision_matrix[cost_criteria] = 1 - scaler.fit_transform(decision_matrix[cost_criteria])

    df_feature_importances["Average_Weight"] = df_feature_importances.mean(axis=1)
    weights = df_feature_importances["Average_Weight"].to_dict()

    for feature in selected_features:
        decision_matrix[feature] *= weights[feature]

    return decision_matrix, selected_features

decision_matrix, selected_features = prepare_decision_matrix(data, df_feature_importances)


<h2>TOPSIS: Technique for Order Preference by Similarity to Ideal Solution</h2>

<p>
    TOPSIS is a widely used Multi-Criteria Decision Making (MCDM) method that evaluates and ranks alternatives
    based on their relative distance from an ideal best and worst solution. It helps in making objective decisions
    by considering both beneficial and non-beneficial criteria.
</p>

<h3>Methodology:</h3>
<ul>
    <li><b>Construct a Decision Matrix:</b> Each vendor's performance is recorded across multiple criteria.</li>
    <li><b>Identify Ideal and Worst Solutions:</b> The best and worst values for each criterion are determined.</li>
    <li><b>Calculate Distances:</b> The Euclidean distance of each alternative from the ideal best and worst solutions is computed.</li>
    <li><b>Compute TOPSIS Score:</b> The score is derived as the ratio of distance from the worst solution to the total distance.</li>
    <li><b>Rank Alternatives:</b> Vendors are ranked in descending order based on their TOPSIS scores.</li>
</ul>

<p>
    A higher TOPSIS score indicates a vendor is closer to the ideal solution, making it a preferable choice.
    This method ensures a balanced selection by considering both the best and worst-case scenarios in decision-making.
</p>


In [75]:
def topsis(decision_matrix, selected_features):
    matrix = decision_matrix[selected_features].values.astype(float)
    ideal_best = np.max(matrix, axis=0)
    ideal_worst = np.min(matrix, axis=0)
    distance_best = np.linalg.norm(matrix - ideal_best, axis=1)
    distance_worst = np.linalg.norm(matrix - ideal_worst, axis=1)
    return distance_worst / (distance_best + distance_worst)

decision_matrix['TOPSIS_Score'] = topsis(decision_matrix, selected_features)
print(decision_matrix[['vendor code', 'TOPSIS_Score']]
      .sort_values(by='TOPSIS_Score', ascending=False)
      .head(10)
      .to_string(index=False))


 vendor code  TOPSIS_Score
    103159.0      0.998760
    101997.0      0.507537
    101868.0      0.053586
      3028.0      0.050175
    700112.0      0.044341
      3365.0      0.041179
    302531.0      0.041070
    302562.0      0.039873
    700112.0      0.027401
    302531.0      0.018667



<h2>VIKOR: Multi-Criteria Optimization and Compromise Ranking</h2>

<p>
    VIKOR is a powerful Multi-Criteria Decision Making (MCDM)
    method designed to identify the best compromise solution among multiple alternatives. It is particularly useful
    when decision-makers need to balance conflicting criteria.
</p>

<h3>Methodology:</h3>
<ul>
    <li><b>Determine Best and Worst Values:</b> Identify the best and worst performance for each criterion.</li>
    <li><b>Compute Utility and Regret Measures:</b>
        <ul>
            <li><b>Si (Utility Measure):</b> Aggregates the deviations of an alternative from the best criterion values.</li>
            <li><b>Ri (Regret Measure):</b> Captures the worst deviation among all criteria for a given alternative.</li>
        </ul>
    </li>
    <li><b>Calculate the VIKOR Score:</b> A weighted sum of Si and Ri is used to determine the ranking.</li>
    <li><b>Rank Alternatives:</b> Lower VIKOR scores indicate better alternatives that offer an optimal trade-off between competing criteria.</li>
</ul>

<p>
    VIKOR is particularly effective in scenarios where a compromise solution is needed rather than a strict optimal solution.
    It provides a balanced approach by considering both the best and worst performance levels in decision-making.
</p>



In [76]:
def vikor(decision_matrix, selected_features):
    matrix = decision_matrix[selected_features].values
    f_best = np.min(matrix, axis=0)
    f_worst = np.max(matrix, axis=0)
    Si = np.sum((matrix - f_best) / (f_worst - f_best), axis=1)
    Ri = np.max((matrix - f_best) / (f_worst - f_best), axis=1)
    S_best, S_worst = np.min(Si), np.max(Si)
    R_best, R_worst = np.min(Ri), np.max(Ri)
    return 0.5 * ((Si - S_best) / (S_worst - S_best)) + 0.5 * ((Ri - R_best) / (R_worst - R_best))

decision_matrix['VIKOR_Score'] = vikor(decision_matrix, selected_features)
print(decision_matrix[['vendor code', 'VIKOR_Score']].sort_values(by='VIKOR_Score', ascending=False).head(10).to_string(index=False))


 vendor code VIKOR_Score
    103159.0         1.0
    103159.0    0.929756
    302531.0    0.926874
    302531.0    0.926639
    101997.0    0.913073
    103491.0    0.906661
      6572.0    0.898836
    102141.0    0.897866
    103159.0    0.897586
    190482.0    0.895941



<h2>Analytic Hierarchy Process (AHP)</h2>

<p>
    AHP is a decision-making method that ranks alternatives based on weighted criteria.
    It uses pairwise comparisons to derive priority weights and ensures consistency in judgments.
</p>

<h3>Key Steps:</h3>
<ul>
    <li>Construct a pairwise comparison matrix.</li>
    <li>Compute priority weights using eigenvalues.</li>
    <li>Check consistency (CR &lt; 0.1 preferred).</li>
    <li>Calculate and normalize AHP scores.</li>
</ul>

<p>
    This method is useful for selecting the best vendor based on multiple attributes.
</p>



In [77]:
def compute_pairwise_matrix(weights):
    n = len(weights)
    return np.array([[weights[i] / weights[j] if weights[j] != 0 else 1 for j in range(n)] for i in range(n)])

def compute_priority_weights(pairwise_matrix):
    eigenvalues, eigenvectors = eig(pairwise_matrix)
    principal_eigenvector = eigenvectors[:, np.argmax(eigenvalues)].real
    return principal_eigenvector / principal_eigenvector.sum()

def check_consistency(pairwise_matrix, priority_weights):
    n = len(priority_weights)
    lambda_max = np.max(eig(pairwise_matrix)[0].real)
    consistency_index = (lambda_max - n) / (n - 1) if n > 1 else 0
    random_index = {1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12, 6: 1.24, 7: 1.32,
                    8: 1.41, 9: 1.45, 10: 1.49}.get(n, 1.49)
    consistency_ratio = consistency_index / random_index if random_index else 0
    if consistency_ratio > 0.1:
        print(f"⚠️ Warning: Consistency Ratio = {consistency_ratio:.3f} (> 0.1), weights may need adjustment!")

def calculate_ahp_scores(data, selected_features, priority_weights):
    decision_matrix = data[['vendor code'] + selected_features].copy()
    for feature, weight in zip(selected_features, priority_weights):
        decision_matrix[feature] *= weight
    decision_matrix['AHP_Score'] = decision_matrix[selected_features].sum(axis=1)
    decision_matrix["AHP_Score_Normalized"] = (
        decision_matrix["AHP_Score"] - decision_matrix["AHP_Score"].min()
    ) / (decision_matrix["AHP_Score"].max() - decision_matrix["AHP_Score"].min())
    return decision_matrix[['vendor code', 'AHP_Score_Normalized']].sort_values(by="AHP_Score_Normalized", ascending=False)

def ahp_ranking(data, df_feature_importances):
    selected_features = df_feature_importances[df_feature_importances.sum(axis=1) > 0].index.tolist()
    weights = df_feature_importances.loc[selected_features].mean(axis=1).values
    pairwise_matrix = compute_pairwise_matrix(weights)
    priority_weights = compute_priority_weights(pairwise_matrix)
    check_consistency(pairwise_matrix, priority_weights)
    return calculate_ahp_scores(data, selected_features, priority_weights)

ranked_vendors = ahp_ranking(data, df_feature_importances)
print(ranked_vendors.head(10).to_string(index=False))

 vendor code AHP_Score_Normalized
    103159.0                  1.0
    101997.0             0.623734
    302562.0             0.047029
    302531.0             0.045608
      3365.0             0.043022
    101868.0             0.024576
      3028.0             0.023486
    302531.0             0.021301
    302531.0             0.021106
    700112.0             0.020814


<h1 style="text-align: center; font-size: 36px; color: #d9534f;">Final Note</h1>

<h1>Interpreting and Utilizing MCDM Scores</h1>

<p>
    Now that we have obtained the final scores from all Multi-Criteria Decision Making (MCDM) algorithms,
    these rankings can be effectively used for decision-making. Each algorithm provides a different
    perspective on vendor evaluation:
</p>

<ul>
    <li><b>TOPSIS</b> ranks vendors based on their closeness to an ideal solution.</li>
    <li><b>VIKOR</b> balances compromise solutions, highlighting trade-offs.</li>
    <li><b>AHP</b> determines rankings based on a structured hierarchical weighting approach.</li>
</ul>

<p>
    By analyzing these scores collectively, decision-makers can:
</p>

<ul>
    <li>✔ Identify the top-performing vendors across multiple ranking techniques.</li>
    <li>✔ Compare rankings from different algorithms to ensure robustness.</li>
    <li>✔ Use the results to make informed procurement, partnership, or investment decisions.</li>
</ul>

<p>
    In practical applications, the rankings can be integrated into dashboards,
    automated procurement systems, or decision support tools to streamline
    vendor selection and optimize resource allocation.
</p>
